In [1]:
import os
os.chdir('../')

In [2]:
!pip install -U torch
!pip install medialpy
!pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!pip install tensorflow-addons


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


# Import modules and load data

In [4]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [5]:
%reload_ext autoreload

In [6]:
from itertools import combinations
import json
import os
import shutil
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel, AutoTokenizer, TFAutoModel
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import CLIP_model
import CLIP_data_load

2023-08-23 19:05:55.065238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information s

In [7]:
# Hyperparameters
train_percentage = 0.8
test_percentage = 0.2
validation_percentage = 0.15
latent_dim_imgs = 1024
latent_dim_text = 768
latent_dim_common = 512
batch_size = 128
SEED = 116
img_shape = (128,128,3)
text_input_shape = (128)
bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased
bert_model_name = "medicalai/ClinicalBERT"
max_len = 128
model_name = 'clip_no_stopwords_our_loss'

# Paths
all_images_path = 'dataset/resized_train/'
project_location = '.'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
model_loc = f'{project_location}/weights/{model_name}.h5'

In [8]:
run = wandb.init(project="clip",
           entity='calonca',
           name=model_name,
           config = {
              'batch_size':batch_size,
              'latent_dim_common':latent_dim_common,
              'latent_dim_text':latent_dim_text,
              'latent_dim_imgs':latent_dim_imgs,
              'img_shape':img_shape,
              'text_input_shape':text_input_shape,
              'model_name': model_name,
              'bert_model_name': bert_model_name,
           })

wandb: Currently logged in as: pablo-giaccaglia (calonca). Use `wandb login --relogin` to force relogin


## **PRE TRAINED BERT MODEL**

## **PRE TRAINED FEATURE EXTRACTION NETWORK**

## **CLIP MODEL**

# Data preparation

In [9]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

concepts = concept_df.set_index('concept').T.to_dict('list')
concept_id_df.cuis = concept_id_df.cuis.apply(lambda x: x.split(';'))
concept_id_df['concepts'] = concept_id_df.cuis.apply(lambda cuis: {concepts[cui][0] for cui in cuis})

captionsDF = pd.merge(captionsDF,concept_id_df, on="ID")
captionsDF.ID = captionsDF.ID.apply(lambda x: x+'.jpg')

# **Data generation**

In [10]:
train_val, test = train_test_split(captionsDF, test_size = test_percentage, shuffle = True, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, random_state = SEED)

In [11]:
words_to_exclude = [
    'show',
    'arrow',
    'image',
    'scan',
    'demonstrate',
    'patient',
    'compute',
    'reveal',
    'imaging',
    'see',
    'level',
    'case'
    'note',
    'month',
    'within',
    'indicate',
    'also',
    'without',
    'year',
    'day',
    'followup',
    'follow',
    'finding',
    'well',
    'along',
    'figure',
    'week',
    'obtain',
    'take',
    'main',
    'type',
    'group',
    'defect',
    'admission',
    'leave'
]

In [12]:
train = CLIP_data_load.captions_clean(train, words_to_remove=words_to_exclude)
val = CLIP_data_load.captions_clean(val,  words_to_remove=words_to_exclude)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
#tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
max_len_concepts = 80
max_len_captions = 128
trainList = CLIP_data_load.paths_captions_concepts_emb_list(train,  all_images_path,tokenizer=tokenizer,max_len_concepts=max_len_concepts, max_len_captions=max_len_captions, remove_images_threshold = 0)
valList = CLIP_data_load.paths_captions_concepts_emb_list(val, all_images_path,tokenizer=tokenizer,max_len_concepts=max_len_concepts, max_len_captions=max_len_captions,  remove_images_threshold = 0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9993/9993 [00:06<00:00, 1519.41it/s]


In [14]:
train_gen = CLIP_data_load.ClipBaseGenerator(channels_first = False, preprocessing_function = None, data = trainList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)
val_gen = CLIP_data_load.ClipBaseGenerator(channels_first = False, preprocessing_function = None, data = valList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)

## **Train CLIP**

In [15]:
len(next(iter(train_gen)))

2023-08-23 19:07:18.192784: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46165 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:c1:00.0, compute capability: 8.6


2

In [16]:
fen_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
    include_top = False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)

caption_input_shape = (128)
concepts_input_shape = (80)
#fen_model = TFAutoModel.from_pretrained("ngctnnnn/lvmmed_rn50",  from_pt=True)
#img_shape=(3,224,224)
#bert_model = TFDistilBertModel.from_pretrained(bert_model_name)
bert_model = TFAutoModel.from_pretrained("medicalai/ClinicalBERT", from_pt=True)

model = CLIP_model.get_clip_fusion_model(
    image_input_shape = img_shape,
    caption_input_shape=caption_input_shape,
    concepts_input_shape= concepts_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True,
    loss = CLIP_model.get_hybrid_loss(temperature_base_loss=0.8, temperature_custom_loss=0.5, weight=0.25, verbose_clip_loss=False)
    custom_metric_tracker = CLIP_model.LastValueMetricClipLoss()
    lr_image_encoder = 1e-3,
    lr_text_encoder = 1e-5,
    lr_head = 1e-4,
    projector_dropout_rate = 0.1
)

2023-08-23 19:07:22.284020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to

In [17]:
model.layers

In [18]:
img_shape

(128, 128, 3)

In [19]:
model.summary()

Model: "clip_fusion_with_custom_metric"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 80)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 80)]                 0         []                            
                                                                     

In [20]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights=True)

In [21]:
from wandb.keras import WandbCallback
wandb_callback = WandbCallback(monitor='loss',
                               log_batch_frequency=10,
                               save_model = False)

In [22]:
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 100, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/100


2023-08-23 19:07:28.527920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-23 19:07:31.964804: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xcfe9f180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-23 19:07:31.964845: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-08-23 19:07:31.969621: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-23 19:07:32.105818: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


441/441 [==============================] - ETA: 0s - loss: 22.1417 - clip_loss: 4.8395 - clip_caption_loss: 4.8397 - clip_image_loss: 4.8392

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
!mkdir -p {project_location}/weights
model.save(f"{project_location}/weights/{model_name}.h5")

In [ ]:
artifact = wandb.Artifact(model_name, type='model')
artifact.add_file(f"{project_location}/weights/{model_name}.h5")
run.log_artifact(artifact)
run.finish()